# 1、ConversationTokenBufferMemory的使用

举例1:

In [ ]:
# 1. 导入相关包
from langchain.memory import ConversationTokenBufferMemory
from langchain_openai import ChatOpenAI

# 2. 创建大模型
# Note: For this example to work correctly, the LLM must be available and configured.
llm = ChatOpenAI(model="gpt-4o-mini")

# 3. 定义ConversationTokenBufferMemory对象
#    llm: 用于计算token数
#    max_token_limit: 设置token上限
memory = ConversationTokenBufferMemory(
    llm=llm,
    max_token_limit=10  # 设置token上限，默认2000 (This value is too small for a real conversation, just for demonstration)
)

# 添加对话
memory.save_context({"input": "你好吗?"}, {"output": "我很好, 谢谢!"})
memory.save_context({"input": "今天天气如何?"}, {"output": "晴天, 25度"})

# 查看当前记忆
print(memory.load_memory_variables({}))

# 2、ConversationSummaryMemory的使用

举例1: 如果实例化`ConversationSummaryMemory`前，没有历史消息，可以使用构造方法实例化

In [ ]:
# 1. 导入相关包
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import ChatOpenAI

# 2. 创建大模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 3. 定义ConversationSummaryMemory对象
memory = ConversationSummaryMemory(llm=llm)

# 4. 存储消息
memory.save_context({"input": "你好"}, {"output": "怎么了"})
memory.save_context({"input": "你是谁"}, {"output": "我是AI助手小智"})
memory.save_context({"input": "初次对话, 你能介绍一下你自己吗?"}, {"output": "当然可以了。我是一个无所不能的小智。"})

# 5. 读取消息 (总结后的)
print(memory.load_memory_variables({}))

举例2: 如果实例化`ConversationSummaryMemory`前, 已经有历史消息, 可以调用`from_messages()`实例化

In [ ]:
# 1. 导入相关包
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain_openai import ChatOpenAI

# 2. 定义ChatMessageHistory对象 (Note: llm is created here, not ChatMessageHistory)
llm = ChatOpenAI(model="gpt-4o-mini")

# 3. 假设原始消息
history = ChatMessageHistory()
history.add_user_message("你好, 你是谁?")
history.add_ai_message("我是AI助手小智")

# 4. 创建ConversationSummaryMemory的实例
memory = ConversationSummaryMemory.from_messages(
    llm=llm,
    #是生成摘要的原材料，保留完整对话供必要时溯洄，当新增对话时，LLM需要结合原始历史生成新摘要
    chat_memory=history,
)

print(memory.load_memory_variables({}))

memory.save_context(inputs={"human":"我的名字叫小明"}, outputs={"ai": "很高兴认识你"})

print(memory.load_memory_variables({}))
print(memory.chat_memory.messages)

# 3、ConversationSummaryBufferMemory的使用

举例1:

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory

# 获取大型模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 实例化ConversationSummaryBufferMemory
memory = ConversationSummaryBufferMemory(
    llm = llm,
    max_token_limit=40, # 控制缓冲器区的大小
    return_messages=True,
)

# 向memory中存储信息
memory.save_context(inputs={"input":"你好, 我的名字叫小明"}, outputs={"output":"很高兴认识你"})
memory.save_context(inputs={"input":"李白是哪个朝代的诗人"}, outputs={"output":"李白是唐朝的诗人"})
memory.save_context(inputs={"input":"唐宋八大家里有苏轼吗？"}, outputs={"output":"有"})

print(memory.load_memory_variables({}))

print("\n")

print(memory.chat_memory.messages)

In [ ]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.llm import LLMChain
# 1、初始化大语言模型
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    max_tokens=500
)
#2、定义提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是电商客服助手，用中文友好回复用户问题。保持专业但亲切的语气。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

# 3、创建带摘要缓冲的记忆系统
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=400,
    memory_key="chat_history",
    return_messages=True
)
#4、创建对话链
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
)
#5、模拟多轮对话
dialogue = [
    ("你好，我想查询订单12345的状态", None),
    ("这个订单是上周五下的", None),
    ("我现在急着用，能加急处理吗", None),
    ("等等，我可能记错订单号了，应该是12346", None),
    ("对了，你们退货政策是怎样的", None)
]
#6、执行对话
for user_input, _ in dialogue:
    response = chain.invoke({"input": user_input})
    print(f"用户: {user_input}")
    print(f"客服: {response['text']}\n")

# 7、查看当前记忆状态
print("\n=== 当前记忆内容 ===")
print(memory.load_memory_variables({}))

# 4、ConversationEntityMemory的使用（了解）

In [ ]:
from langchain.chains.conversation.base import LLMChain
from langchain.memory import ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain_openai import ChatOpenAI
# 初始化大语言模型
llm = ChatOpenAI(model_name='gpt-4o-mini', temperature=0)
# 使用LangChain为实体记忆设计的预定义模板
prompt = ENTITY_MEMORY_CONVERSATION_TEMPLATE
# 初始化实体记忆
memory = ConversationEntityMemory(llm=llm)
# 提供对话链
chain = LLMChain(
    llm=llm,
    prompt=ENTITY_MEMORY_CONVERSATION_TEMPLATE,
    memory=ConversationEntityMemory(llm=llm),
#verbose=True, # 设置为True可以看到链的详细推理过程
)
#进行几轮对话，记忆组件会在后台自动提取和存储实体信息
chain.invoke(input="你好，我叫蜘蛛侠。我的好朋友包括钢铁侠、美国队长和绿巨人。")
chain.invoke(input="我住在纽约。")
chain.invoke(input="我使用的装备是由斯塔克工业提供的。")
# 查询记忆体中存储的实体信息
print("\n当前存储的实体信息:")
print(chain.memory.entity_store.store)
# 基于记忆进行提问
answer = chain.invoke(input="你能告诉我蜘蛛侠住在哪里以及他的好朋友有哪些吗？")
print("\nAI的回答:")
print(answer)

# 3.5 ConversationKGMemory(了解)

In [ ]:
# 1. 导入相关包
from langchain.memory import ConversationKGMemory
from langchain.chat_models import ChatOpenAI

# 2. 定义LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3. 定义ConversationKGMemory对象
memory = ConversationKGMemory(llm=llm)

# 4. 保存会话
memory.save_context({"input": "向山姆问好"}, {"output": "山姆是谁"})
memory.save_context({"input": "山姆是我的朋友"}, {"output": "好的"})

# 5. 查询会话
memory.load_memory_variables({"input": "山姆是谁"})